In [1]:
import sys, os, re
from typing import *
from typing import Any, Iterator
from dataclasses import dataclass
sys.path = (["../src/", "../"] if re.match(r"^(\w\:\\)|(/)", os.getcwd()) else []) + sys.path
import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger, QubxLogConfig
from qubx.core.basics import DataType, Instrument, Signal
from qubx.core.interfaces import IStrategy, IStrategyContext, TriggerEvent
from qubx.core.series import Quote, Bar
from qubx.pandaz.utils import *
from qubx.core.account import BasicAccountProcessor
from qubx.core.context import StrategyContext
from qubx.core.loggers import InMemoryLogsWriter, StrategyLogging

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.data.helpers import loader
from qubx.data.readers import DataReader, DataTransformer, AsPandasFrame, AsTimestampedRecords, RestoreTicksFromOHLC
from qubx.backtester.simulated_data import IteratedDataStreamsSlicer, IterableSimulationData

from qubx.backtester.simulator import simulate, SimulatedTrading, SimulatedExchange

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [2]:
s1 = lookup.find_symbol("BINANCE.UM", "BTCUSDT")
s2 = lookup.find_symbol("BINANCE.UM", "ETHUSDT")
s3 = lookup.find_symbol("BINANCE.UM", "LTCUSDT")

In [3]:
ld = loader("BINANCE.UM", "1h", source="csv::../tests/data/csv_1h", n_jobs=1)
# ld = loader("BINANCE.UM", "1h", source="mqdb::xlydian-data", n_jobs=1)

### <font color='#707070'><s>Test Old DataLoader</s></font>

In [77]:
q = SimulatedDataQueue()
l1 = DataLoader(RestoreTicksFromOHLC(), ld, s1, warmup_period="1d", timeframe="1h")
l2 = DataLoader(RestoreTicksFromOHLC(), ld, s2, warmup_period="1d", timeframe="1h")
q += l1
q += l2

In [78]:
k = 0
itr = q.create_iterable("2023-07-01", "2023-07-02")
for t in itr:
    print(f"{pd.Timestamp(t[2].time, 'ns')}     \t {t[0].symbol}\t{t[2]} / {t[1]}")
    if k == 10:
        print("+ " * 10)
        q += (l3:=DataLoader(RestoreTicksFromOHLC(), ld, s3, warmup_period="1d", timeframe="1h"))

    if k == 30:
        print("- " * 10)
        # q -= l3
    k+=1

2023-06-30 00:00:01     	 BTCUSDT	[2023-06-30T00:00:01.000000000]	30439.90000 (1000000000.0) | 30439.90000 (1000000000.0) / hist_ohlc
2023-06-30 00:00:01     	 ETHUSDT	[2023-06-30T00:00:01.000000000]	1851.18000 (1000000000.0) | 1851.18000 (1000000000.0) / hist_ohlc
2023-06-30 00:24:00     	 BTCUSDT	[2023-06-30T00:24:00.000000000]	30500.00000 (1000000000.0) | 30500.00000 (1000000000.0) / hist_ohlc
2023-06-30 00:24:00     	 ETHUSDT	[2023-06-30T00:24:00.000000000]	1856.26000 (1000000000.0) | 1856.26000 (1000000000.0) / hist_ohlc
2023-06-30 00:36:00     	 BTCUSDT	[2023-06-30T00:36:00.000000000]	30385.00000 (1000000000.0) | 30385.00000 (1000000000.0) / hist_ohlc
2023-06-30 00:36:00     	 ETHUSDT	[2023-06-30T00:36:00.000000000]	1846.64000 (1000000000.0) | 1846.64000 (1000000000.0) / hist_ohlc
2023-06-30 00:59:59     	 BTCUSDT	[2023-06-30T00:59:59.000000000]	30431.30000 (1000000000.0) | 30431.30000 (1000000000.0) / hist_ohlc
2023-06-30 00:59:59     	 ETHUSDT	[2023-06-30T00:59:59.000000000]	18

---

### <font color='#00ff00'>Impl. multi-symbol</font>

In [4]:
@dataclass
class DummyTimeEvent:
    time: pd.Timestamp
    data: str

    @staticmethod
    def from_dict(data: dict[str | pd.Timedelta, str], start: str) -> list["DummyTimeEvent"]:
        _t0 = pd.Timestamp(start)
        return [DummyTimeEvent((_t0 + pd.Timedelta(t)).as_unit("ns").asm8.item(), d) for t, d in data.items()]

    @staticmethod
    def from_seq(start: str, n: int, ds: str, pfx: str) -> list["DummyTimeEvent"]:
        return DummyTimeEvent.from_dict({s * pd.Timedelta(ds): pfx for s in range(n+1)}, start)

    def __repr__(self) -> str:
        return f"{pd.Timestamp(self.time, 'ns')} -> ({self.data})"

#### - First impl -

In [5]:
slicer = IteratedDataStreamsSlicer()

data1 = [
    DummyTimeEvent.from_seq("2020-01-01 00:00", 10, "1Min", "A1"),
    DummyTimeEvent.from_seq("2020-01-01 00:11", 10, "1Min", "A2"),
]

data2 = [
    DummyTimeEvent.from_seq("2020-01-01 00:05", 10, "1Min", "B1"),
    DummyTimeEvent.from_seq("2020-01-01 00:16", 10, "1Min", "B2"),
]

data3 = [
    DummyTimeEvent.from_seq("2020-01-01 00:08", 10, "1Min", "C1"),
    DummyTimeEvent.from_seq("2020-01-01 00:19", 10, "1Min", "C2"),
]

slicer += {
    'it3': iter(data3),
    'it2': iter(data2),
    'it1': iter(data1),
}

r = []
for t in slicer:
    if not t: continue
    print(f"{pd.Timestamp(t[2].time, 'ns')} | id={t[0]} | {t[2].data}")
    r.append(t[2].data)

2020-01-01 00:00:00 | id=it1 | A1
2020-01-01 00:01:00 | id=it1 | A1
2020-01-01 00:02:00 | id=it1 | A1
2020-01-01 00:03:00 | id=it1 | A1
2020-01-01 00:04:00 | id=it1 | A1
2020-01-01 00:05:00 | id=it1 | A1
2020-01-01 00:05:00 | id=it2 | B1
2020-01-01 00:06:00 | id=it1 | A1
2020-01-01 00:06:00 | id=it2 | B1
2020-01-01 00:07:00 | id=it1 | A1
2020-01-01 00:07:00 | id=it2 | B1
2020-01-01 00:08:00 | id=it1 | A1
2020-01-01 00:08:00 | id=it2 | B1
2020-01-01 00:08:00 | id=it3 | C1
2020-01-01 00:09:00 | id=it1 | A1
2020-01-01 00:09:00 | id=it2 | B1
2020-01-01 00:09:00 | id=it3 | C1
2020-01-01 00:10:00 | id=it1 | A1
2020-01-01 00:10:00 | id=it2 | B1
2020-01-01 00:10:00 | id=it3 | C1
2020-01-01 00:11:00 | id=it1 | A2
2020-01-01 00:11:00 | id=it2 | B1
2020-01-01 00:11:00 | id=it3 | C1
2020-01-01 00:12:00 | id=it1 | A2
2020-01-01 00:12:00 | id=it2 | B1
2020-01-01 00:12:00 | id=it3 | C1
2020-01-01 00:13:00 | id=it1 | A2
2020-01-01 00:13:00 | id=it2 | B1
2020-01-01 00:13:00 | id=it3 | C1
2020-01-01 00:

#### <font color='#00ff00'>- New impl test -</font>

In [6]:
slicer = IteratedDataStreamsSlicer()

slicer += {
    "set.A": iter([DummyTimeEvent.from_seq("2020-01-01 00:00", 10, "1Min", "A1")]),
    "set.C": iter([DummyTimeEvent.from_seq("2020-01-01 00:01", 10, "1Min", "C1")]),
    "set.D": iter([DummyTimeEvent.from_seq("2020-01-01 00:03", 10, "1Min", "D1")]),
    "set.E": iter([DummyTimeEvent.from_seq("2020-01-01 00:03", 10, "1Min", "E1")]),
    "set.B": iter([DummyTimeEvent.from_seq("2020-01-01 00:05", 10, "1Min", "B1")]),
}

r = []
for t in slicer:
    if not t:
        continue
    print(f"{pd.Timestamp(t[2].time, 'ns')} | id={t[0]} | {t[2].data}")
    r.append(t[2].data)

2020-01-01 00:00:00 | id=set.A | A1
2020-01-01 00:01:00 | id=set.A | A1
2020-01-01 00:01:00 | id=set.C | C1
2020-01-01 00:02:00 | id=set.A | A1
2020-01-01 00:02:00 | id=set.C | C1
2020-01-01 00:03:00 | id=set.A | A1
2020-01-01 00:03:00 | id=set.C | C1
2020-01-01 00:03:00 | id=set.D | D1
2020-01-01 00:03:00 | id=set.E | E1
2020-01-01 00:04:00 | id=set.A | A1
2020-01-01 00:04:00 | id=set.C | C1
2020-01-01 00:04:00 | id=set.D | D1
2020-01-01 00:04:00 | id=set.E | E1
2020-01-01 00:05:00 | id=set.A | A1
2020-01-01 00:05:00 | id=set.C | C1
2020-01-01 00:05:00 | id=set.D | D1
2020-01-01 00:05:00 | id=set.E | E1
2020-01-01 00:05:00 | id=set.B | B1
2020-01-01 00:06:00 | id=set.A | A1
2020-01-01 00:06:00 | id=set.C | C1
2020-01-01 00:06:00 | id=set.D | D1
2020-01-01 00:06:00 | id=set.E | E1
2020-01-01 00:06:00 | id=set.B | B1
2020-01-01 00:07:00 | id=set.A | A1
2020-01-01 00:07:00 | id=set.C | C1
2020-01-01 00:07:00 | id=set.D | D1
2020-01-01 00:07:00 | id=set.E | E1
2020-01-01 00:07:00 | id=set

In [140]:
print(r)

['A1', 'A1', 'C1', 'A1', 'C1', 'A1', 'C1', 'D1', 'E1', 'A1', 'C1', 'D1', 'E1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'A1', 'C1', 'D1', 'E1', 'B1', 'C1', 'D1', 'E1', 'B1', 'D1', 'E1', 'B1', 'D1', 'E1', 'B1', 'B1', 'B1']


#### Add / remove on the fly

In [5]:
slicer = IteratedDataStreamsSlicer()

data1 = [
    DummyTimeEvent.from_seq("2020-01-01 00:00", 10, "1Min", "A1"),
    DummyTimeEvent.from_seq("2020-01-01 00:11", 10, "1Min", "A2"),
]

data2 = [
    DummyTimeEvent.from_seq("2020-01-01 00:05", 10, "1Min", "B1"),
    DummyTimeEvent.from_seq("2020-01-01 00:16", 10, "1Min", "B2"),
]

data3 = [
    DummyTimeEvent.from_seq("2020-01-01 00:08", 10, "1Min", "C1"),
    DummyTimeEvent.from_seq("2020-01-01 00:19", 10, "1Min", "C2"),
]

data4 = [
    DummyTimeEvent.from_seq("2020-01-01 00:10", 10, "1Min", "D1"),
]

slicer += {
    '1': iter(data1),
    '2': iter(data2),
    '3': iter(data3),
}

r, k = [], 0
for t in slicer:
    if not t: continue
    print(f"{k:3d}: {pd.Timestamp(t[2].time, 'ns')} | id={t[0]} | {t[2].data}")
    r.append(t[2].data)
    if k == 3: slicer.remove('1')
    if k == 11: slicer += {'10': iter(data4)}
    k += 1

  0: 2020-01-01 00:00:00 | id=1 | A1
  1: 2020-01-01 00:01:00 | id=1 | A1
  2: 2020-01-01 00:02:00 | id=1 | A1
  3: 2020-01-01 00:03:00 | id=1 | A1
  4: 2020-01-01 00:05:00 | id=2 | B1
  5: 2020-01-01 00:06:00 | id=2 | B1
  6: 2020-01-01 00:07:00 | id=2 | B1
  7: 2020-01-01 00:08:00 | id=2 | B1
  8: 2020-01-01 00:08:00 | id=3 | C1
  9: 2020-01-01 00:09:00 | id=2 | B1
 10: 2020-01-01 00:09:00 | id=3 | C1
 11: 2020-01-01 00:10:00 | id=2 | B1
 12: 2020-01-01 00:10:00 | id=3 | C1
 13: 2020-01-01 00:10:00 | id=10 | D1
 14: 2020-01-01 00:11:00 | id=3 | C1
 15: 2020-01-01 00:11:00 | id=10 | D1
 16: 2020-01-01 00:11:00 | id=2 | B1
 17: 2020-01-01 00:12:00 | id=3 | C1
 18: 2020-01-01 00:12:00 | id=10 | D1
 19: 2020-01-01 00:12:00 | id=2 | B1
 20: 2020-01-01 00:13:00 | id=3 | C1
 21: 2020-01-01 00:13:00 | id=10 | D1
 22: 2020-01-01 00:13:00 | id=2 | B1
 23: 2020-01-01 00:14:00 | id=3 | C1
 24: 2020-01-01 00:14:00 | id=10 | D1
 25: 2020-01-01 00:14:00 | id=2 | B1
 26: 2020-01-01 00:15:00 | id=3 |

In [8]:
print(r)

['A1', 'A1', 'A1', 'A1', 'B1', 'B1', 'B1', 'B1', 'C1', 'B1', 'C1', 'B1', 'C1', 'D1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B1', 'C1', 'D1', 'B2', 'C1', 'D1', 'B2', 'C1', 'D1', 'B2', 'C2', 'D1', 'B2', 'C2', 'D1', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'B2', 'C2', 'C2', 'C2']


### <font color='#ff0050'>Subscriptions blotter</fint>

In [6]:
ld = loader("BINANCE.UM", "1h", source="csv::../tests/data/csv_1h", n_jobs=1)

In [7]:
isd = IterableSimulationData(ld, open_close_time_indent_secs=300)
isd.set_warmup_period(DataType.OHLC['1d'], "24h")

In [8]:
isd.add_instruments_for_subscription(DataType.OHLC['1h'], [s1, s2])
isd.add_instruments_for_subscription(DataType.OHLC['1h'], s3)
isd.add_instruments_for_subscription(DataType.OHLC['4h'], s3)
isd.add_instruments_for_subscription(DataType.OHLC['1d'], s3)
isd.add_instruments_for_subscription(DataType.OHLC_TICKS['4h'], s1)

In [9]:
isd.get_instruments_for_subscription(DataType.OHLC_TICKS['4h'])

[BINANCE.UM:SWAP:BTCUSDT]

In [10]:
isd.get_subscriptions_for_instrument(None) #== list(set([ Subtype.OHLC["1h"], Subtype.OHLC["4h"], Subtype.OHLC["1d"]]))

['ohlc_ticks(4h)', 'ohlc(1h)', 'ohlc(1d)', 'ohlc(4h)']

In [11]:
isd.has_subscription(s3, 'ohlc(4h)')

True

In [12]:
isd.get_subscriptions_for_instrument(s3)

['ohlc(1h)', 'ohlc(1d)', 'ohlc(4h)']

In [49]:
# list(map(Subtype.from_str, [Subtype.OHLC["1h"], Subtype.OHLC["4h"], Subtype.OHLC["1d"]]))

In [13]:
isd._subtyped_fetchers

{'ohlc.1h': ohlc({'timeframe': '1h'}) (---) [BINANCE.UM:BTCUSDT,BINANCE.UM:ETHUSDT,BINANCE.UM:LTCUSDT] :-> RestoredBarsFromOHLC,
 'ohlc.4h': ohlc({'timeframe': '4h'}) (---) [BINANCE.UM:LTCUSDT] :-> RestoredBarsFromOHLC,
 'ohlc.1D': ohlc({'timeframe': '1D'}) (-1 days 00:00:00) [BINANCE.UM:LTCUSDT] :-> RestoredBarsFromOHLC,
 'ohlc_ticks.4h': ohlc({'timeframe': '4h'}) (---) [BINANCE.UM:BTCUSDT] :-> RestoreTicksFromOHLC}

In [ ]:
# isd.remove_instruments_from_subscription(Subtype.OHLC_TICKS['4h'], s1)

In [ ]:
isd.remove_instruments_from_subscription(DataType.ALL, s1)

In [74]:
isd.get_subscriptions_for_instrument(None)
# isd.get_instruments_for_subscription(Subtype.ALL)

['ohlc(1h)']

In [10]:
QubxLogConfig.set_log_level('DEBUG')
qiter = isd.create_iterable("2023-07-01", "2023-07-02")

In [12]:
for d in qiter:
    print(pd.Timestamp(d[2].time, 'ns'), d)

2023-07-01 00:05:00 (CRYPTO:BINANCE.UM:BTCUSDT, 'ohlc', {o:30460.200000 | h:30460.200000 | l:30460.200000 | c:30460.200000 | v:0.000000})
2023-07-01 00:05:00 (CRYPTO:BINANCE.UM:ETHUSDT, 'ohlc', {o:1933.420000 | h:1933.420000 | l:1933.420000 | c:1933.420000 | v:0.000000})
2023-07-01 00:05:00 (CRYPTO:BINANCE.UM:LTCUSDT, 'ohlc', {o:108.650000 | h:108.650000 | l:108.650000 | c:108.650000 | v:0.000000})
2023-07-01 00:05:00 (CRYPTO:BINANCE.UM:LTCUSDT, 'ohlc', {o:108.650000 | h:108.650000 | l:108.650000 | c:108.650000 | v:0.000000})
2023-07-01 00:05:00 (CRYPTO:BINANCE.UM:BTCUSDT, 'quote', [2023-07-01T00:05:00.000000000]	30460.20000 (1000000000.0) | 30460.20000 (1000000000.0))
2023-07-01 00:05:00.100000 (CRYPTO:BINANCE.UM:LTCUSDT, 'ohlc', {o:108.650000 | h:108.650000 | l:108.650000 | c:108.650000 | v:0.000000})
2023-07-01 00:24:00 (CRYPTO:BINANCE.UM:BTCUSDT, 'ohlc', {o:30460.200000 | h:30527.200000 | l:30460.200000 | c:30527.200000 | v:4843.225584})
2023-07-01 00:24:00 (CRYPTO:BINANCE.UM:ETHUS

c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\pandaz\utils.py:490: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result.resample(freq)
c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\pandaz\utils.py:490: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result.resample(freq)
c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\pandaz\utils.py:490: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result.resample(freq)
c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\pandaz\utils.py:490: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result.resample(freq)
c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\pandaz\utils.py:490: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result.resample(freq)


In [ ]:
QubxLogConfig.set_log_level('DEBUG')

isd = IterableSimulationData(ld, open_close_time_indent_secs=1)
isd.set_warmup_period(DataType.OHLC['1h'], "1d")
isd.add_instruments_for_subscription(DataType.OHLC['1h'], [s1, s2])

qiter = isd.create_iterable("2023-07-01", "2023-07-02")

k = 0
for t in qiter:
    print(f"{pd.Timestamp(t[2].time, 'ns')} \t{t[1]} \t {t[0].symbol}\t{t[2]}")
    if k == 10:
        print("+ " * 10)
        isd.add_instruments_for_subscription(DataType.OHLC['1h'], s3)

    if k == 30:
        print("- " * 10)
        # q -= l3
    k+=1

2024-12-03 18:22:50.579 [ 🐞 ] (simulated_data) Preloading initial data for ohlc.1H 2023-07-01 00:00:00 : 2023-07-02 00:00:00 ...
2023-06-30 00:00:01 	hist_ohlc 	 BTCUSDT	{o:30439.900000 | h:30439.900000 | l:30439.900000 | c:30439.900000 | v:0.000000}
2023-06-30 00:00:01 	hist_ohlc 	 ETHUSDT	{o:1851.180000 | h:1851.180000 | l:1851.180000 | c:1851.180000 | v:0.000000}
2023-06-30 00:24:00 	hist_ohlc 	 BTCUSDT	{o:30439.900000 | h:30500.000000 | l:30439.900000 | c:30500.000000 | v:4546.392017}
2023-06-30 00:24:00 	hist_ohlc 	 ETHUSDT	{o:1851.180000 | h:1856.260000 | l:1851.180000 | c:1856.260000 | v:36599.260274}
2023-06-30 00:36:00 	hist_ohlc 	 BTCUSDT	{o:30439.900000 | h:30500.000000 | l:30385.000000 | c:30385.000000 | v:5196.957263}
2023-06-30 00:36:00 	hist_ohlc 	 ETHUSDT	{o:1851.180000 | h:1856.260000 | l:1846.640000 | c:1846.640000 | v:50936.372075}
2023-06-30 00:59:59 	hist_ohlc 	 BTCUSDT	{o:30439.900000 | h:30500.000000 | l:30385.000000 | c:30431.300000 | v:8699.419000}
2023-06-30 0

c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\pandaz\utils.py:490: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result.resample(freq)
c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\pandaz\utils.py:490: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result.resample(freq)
c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\pandaz\utils.py:490: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result.resample(freq)


## <font color='#fff050'>Simulations blotter</font>

In [14]:
class TestA(IStrategy):
    timeframe: str = "1d"

    def on_init(self, ctx: IStrategyContext):
        ctx.set_base_subscription(DataType.OHLC[self.timeframe])

    def on_event(self, ctx: IStrategyContext, event: TriggerEvent) -> Optional[List[Signal]]:
        logger.debug("DDDDD")
        if event.data and event.type == "event":
            signal = event.data.get("order")
            if signal is not None and event.instrument:
                return [event.instrument.signal(signal)]
        return None

In [15]:
ld = loader("BINANCE.UM", "1h", source="csv::../tests/data/csv_1h", n_jobs=1)

In [16]:
strat = TestA(timeframe="1h")
account = BasicAccountProcessor(account_id="Test1", base_currency='USDT', initial_capital=1000)
trading_service = SimulatedTrading(account, "BINANCE.UM", "vip0_usdt")
broker = SimulatedExchange("binance.um", trading_service, ld)

2024-12-10 17:41:57.221 [ 🐞 ] (helpers) TestA new parameters:
	set timeframe <- 1h


1970-01-01 00:00:00.000 [ℹ️] SimulatedExchange.binance.um is initialized


In [17]:
# ctx = StrategyContext(
#     strategy=strat,
#     broker=broker,
#     account=account,
#     instruments=[s1, s2],
#     logging=StrategyLogging(InMemoryLogsWriter("test", "test", "0")),
# )
# ctx.start()

In [ ]:
# broker._data_src.get_instruments_for_subscription(Subtype.ALL)

In [18]:
simulate(
   {'A': TestA(timeframe="1h")}, ld, 1000, ['BINANCE.UM:BTCUSDT', 'BINANCE.UM:ETHUSDT', 'BINANCE.UM:LTCUSDT'],
   "vip0_usdt", "2023-07-01", "2023-07-19", debug="DEBUG"
)

1970-01-01 00:00:00.000 [🐞] TestA new parameters:
	set timeframe <- 1h


2024-12-10 17:42:00.142 [ 🐞 ] (simulator) 2023-07-01 00:00:00 Initiating simulated trading for binance.um for 1000 x 1.0 in USDT...
2023-07-01 00:00:00.000 [ℹ️] SimulatedExchange.binance.um is initialized
2023-07-01 00:00:00.000 [🐞]  | get_subscribed_instruments __all__ -> []
2023-07-01 00:00:00.000 [🐞]  | get_subscriptions None -> []
2023-07-01 00:00:00.000 [🐞]  | get_subscribed_instruments ohlc(1h) -> []
2023-07-01 00:00:00.000 [🐞]  | get_subscribed_instruments None -> []
2023-07-01 00:00:00.000 [🐞]  | get_subscribed_instruments ohlc(1h) -> []
2023-07-01 00:00:00.000 [🐞]  | subscribe: ohlc(1h) -> {BINANCE.UM:SWAP:BTCUSDT, BINANCE.UM:SWAP:ETHUSDT, BINANCE.UM:SWAP:LTCUSDT}


Simulating:   0%|          | 0/100 [00:00<?, ?%/s]

2023-07-01 00:00:00.000 [ℹ️] SimulatedExchange ::: Simulation started at 2023-07-01 :::
2023-07-01 00:00:00.000 [🐞] Preloading initial data for ohlc.1h 2023-07-01 00:00:00 : 2023-07-19 00:00:00 ...
2023-07-01 00:00:00.000 [🐞] (InMemoryCachedReader) Loading all data 2023-07-01 00:00:00 - 2023-07-19 00:00:00 for BTCUSDT 
2023-07-01 00:00:00.000 [🐞] (InMemoryCachedReader) Loading all data 2023-07-01 00:00:00 - 2023-07-19 00:00:00 for ETHUSDT 
2023-07-01 00:00:00.000 [🐞] (InMemoryCachedReader) Loading all data 2023-07-01 00:00:00 - 2023-07-19 00:00:00 for LTCUSDT 
2023-07-01 00:00:01.000 [🐞] Invoking TestA on_fit
2023-07-01 00:00:01.000 [🐞] TestA is fitted
2023-07-01 00:59:59.000 [🐞] DDDDD
2023-07-01 01:59:59.000 [🐞] DDDDD
2023-07-01 02:59:59.000 [🐞] DDDDD
2023-07-01 03:59:59.000 [🐞] DDDDD
2023-07-01 04:59:59.000 [🐞] DDDDD
2023-07-01 05:59:59.000 [🐞] DDDDD
2023-07-01 06:59:59.000 [🐞] DDDDD
2023-07-01 07:59:59.000 [🐞] DDDDD
2023-07-01 08:59:59.000 [🐞] DDDDD
2023-07-01 09:59:59.000 [🐞] DDDDD

2023-07-18 21:59:59.000 [🐞] DDDDD
2023-07-18 22:59:59.000 [🐞] DDDDD
2023-07-18 23:59:59.000 [🐞] DDDDD
2023-07-19 00:59:59.000 [🐞] DDDDD


2023-07-19 00:59:59.000 [ℹ️] SimulatedExchange ::: Simulation finished at 2023-07-19 00:00:00 :::
